## Caching
Caching is a technique used to store frequently accessed data in a temporary storage area, known as a cache, to improve performance and reduce latency. By keeping copies of data that are expensive to retrieve or compute, caching allows systems to serve requests more quickly.

### Types of Caching
1. **In-Memory Caching**: Data is stored in the system's RAM for fast access. Examples include Redis and Memcached.
2. **Disk Caching**: Data is stored on a disk, which is slower than RAM but can hold larger amounts of data. Examples include browser caches and operating system caches.
3. **Distributed Caching**: Data is stored across multiple servers to improve scalability and reliability. Examples include Amazon ElastiCache and Apache Ignite.
### Caching Strategies
- **Time-to-Live (TTL)**: Data is cached for a specific duration, after which it is considered stale and must be refreshed.
- **Least Recently Used (LRU)**: The cache evicts the least recently accessed items when it reaches its capacity.
- **Write-Through**: Data is written to both the cache and the underlying data store simultaneously.
- **Write-Back**: Data is written to the cache first and then asynchronously written to the underlying data store

Django provides default caching mechanisms that can be easily integrated into web applications. It supports various caching backends, including in-memory, file-based, database, and distributed caches.


In [ ]:
# Using Memcached with Django Caching
# settings.py
CACHES = {
    'default': {
        'BACKEND': 'django.core.cache.backends.memcached.MemcachedCache',
        'LOCATION': '127.0.0.1:11211',
    }
}

# Caching with Memcached
import memcache
memcached_client = memcache.Client(['127.0.0.1:11211'])
def get_memcached_data(key):
    return memcached_client.get(key)

def set_memcached_data(key, value, time=0):
    memcached_client.set(key, value, time)


In [ ]:
# Using Redis with Django Caching
# settings.py
CACHES = {
    'default': {
        'BACKEND': 'django_redis.cache.RedisCache',
        'LOCATION': 'redis://127.0.0.1:6379/1',
    }
}

# Caching using Redis
import redis
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)
def get_redis_data(key):
    return redis_client.get(key)

def set_redis_data(key, value, ex=None):
    redis_client.set(key, value, ex)


In [ ]:
# Caching Class-Based Views
from django.shortcuts import render
from django.views.decorators.cache import cache_page
from django.utils.decorators import method_decorator
from django.views import View

@method_decorator(cache_page(60 * 15), name="dispatch") # will cache the entire view for 15 minutes
class MyCachedView(View):
    def get(self, request):
        # View logic here
        return render(request, "template.html")

# For specific methods
class MyView(View):
    @method_decorator(cache_page(60 * 10)) # cache only GET requests for 10 minutes
    def get(self, request):
        # Only GET requests are cached
        return render(request, "template.html")
    

# Caching Method-Based Views
from django.views.decorators.cache import cache_page

@cache_page(60 * 15)  # Cache for 15 minutes
def my_view(request):
    return render(request, "template.html")

In [ ]:
#Using Django's default cache framework
from django.core.cache import cache

def get_cached_data(key):
    return cache.get(key)

def set_cached_data(key, value, timeout=None):
    cache.set(key, value, timeout)

def delete_cached_data(key):
    cache.delete(key)

## Cache Invalidation
Cache invalidation is the process of removing or updating cached data when it becomes stale or outdated. Common strategies include:
- **Time-Based Expiration**: Setting a TTL for cached items.
- **Event-Based Invalidation**: Invalidating cache entries based on specific events, such as data updates.
- **Manual Invalidation**: Explicitly removing or updating cache entries through application logic.

```python
# Manual Cache Invalidation
from django.core.cache import cache
from django.db.models.signals import post_save, post_delete
from django.dispatch import receiver
from .models import MyModel

@receiver(post_save, sender=MyModel)
def invalidate_cache_on_save(sender, instance, **kwargs):
    cache_key = f"model_{instance.id}"
    cache.delete(cache_key)

@receiver(post_delete, sender=MyModel)
def invalidate_cache_on_delete(sender, instance, **kwargs):
    cache_key = f"model_{instance.id}"
    cache.delete(cache_key)

# Custom cache key generation
def get_cache_key(model_name, object_id):
    return f"{model_name}_{object_id}"

# Invalidate multiple keys
def invalidate_model_cache(model_name, object_ids):
    keys = [get_cache_key(model_name, oid) for oid in object_ids]
    cache.delete_many(keys)
```

## Limitations and Best Practices
While caching can significantly improve performance, it also has limitations:
- **Stale Data**: Cached data may become outdated, leading to inconsistencies.
- **Cache Size**: Limited cache size may lead to frequent evictions.
- **Complexity**: Implementing caching strategies can add complexity to the application.

Best practices include:
- Regularly monitor cache performance and hit rates.
- Use appropriate cache backends based on application needs.
- Implement effective cache invalidation strategies to maintain data consistency.